In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

import pickle
file = open('wdbc.pkl','rb')
data = pickle.load(file)
data['prediction']=0
data.head()

,id,malignant,radius_0,texture_0,perimeter_0,area_0,smoothness_0,compactness_0,concavity_0,concave points_0,...,texture_2,perimeter_2,area_2,smoothness_2,compactness_2,concavity_2,concave points_2,symmetry_2,fractal dimension_2,prediction
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [10]:
from sklearn.ensemble import RandomForestClassifier
features = ['radius_0', 'texture_0', 'perimeter_0', 'area_0',
       'smoothness_0', 'compactness_0', 'concavity_0', 'concave points_0',
       'symmetry_0', 'fractal dimension_0', 'radius_1', 'texture_1',
       'perimeter_1', 'area_1', 'smoothness_1', 'compactness_1', 'concavity_1',
       'concave points_1', 'symmetry_1', 'fractal dimension_1', 'radius_2',
       'texture_2', 'perimeter_2', 'area_2', 'smoothness_2', 'compactness_2',
       'concavity_2', 'concave points_2', 'symmetry_2', 'fractal dimension_2']
results = []
sample_leaf_options = list(range(1, 50, 3))
n_estimators_options = list(range(1, 100, 5))

#split the data set into training and validation data
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data[features],data['malignant'],test_size=0.25, random_state=0)

for leaf_size in sample_leaf_options:
    for n_estimators_size in n_estimators_options:
        alg = RandomForestClassifier(max_features=0.5, min_samples_leaf=leaf_size, n_estimators=n_estimators_size, random_state=50)
        alg.fit(x_train, y_train)
        predict = alg.predict(x_test)
        # 用一个三元组，分别记录当前的 min_samples_leaf，n_estimators， 和在测试数据集上的精度
        results.append((leaf_size, n_estimators_size, (y_test == predict).mean()))
        # 真实结果和预测结果进行比较，计算准确率
        #print((y_test == predict).mean())
print(max(results, key=lambda x: x[2]))

(7, 56, 0.9790209790209791)


In [11]:
cm = confusion_matrix(y_test,predict)
se = cm[1,1]/(cm[1,1]+cm[1,0])
sp = cm[0,0]/(cm[0,0]+cm[0,1])
print(cm,se,sp)

[[85  5]
 [ 2 51]] 0.9622641509433962 0.9444444444444444


In [13]:
sorted(zip(map(lambda x: round(x, 4), alg.feature_importances_), features), reverse=True)

[(0.3409, 'perimeter_2'),
 (0.2156, 'concave points_2'),
 (0.2128, 'concave points_0'),
 (0.1137, 'radius_2'),
 (0.0627, 'area_2'),
 (0.0221, 'concavity_0'),
 (0.0134, 'perimeter_0'),
 (0.0059, 'concavity_2'),
 (0.0026, 'texture_0'),
 (0.0021, 'area_0'),
 (0.0014, 'radius_0'),
 (0.0012, 'smoothness_2'),
 (0.0011, 'texture_2'),
 (0.001, 'concave points_1'),
 (0.0008, 'area_1'),
 (0.0007, 'compactness_2'),
 (0.0005, 'symmetry_0'),
 (0.0004, 'symmetry_1'),
 (0.0003, 'smoothness_0'),
 (0.0003, 'compactness_0'),
 (0.0002, 'texture_1'),
 (0.0002, 'perimeter_1'),
 (0.0, 'symmetry_2'),
 (0.0, 'smoothness_1'),
 (0.0, 'radius_1'),
 (0.0, 'fractal dimension_2'),
 (0.0, 'fractal dimension_1'),
 (0.0, 'fractal dimension_0'),
 (0.0, 'concavity_1'),
 (0.0, 'compactness_1')]